In [1]:
import sys
import os

# Add parent directory to sys.path
sys.path.append(os.path.abspath(".."))


In [2]:
from models.spin_system import SpinSystem

import pandas as pd
import numpy as np
import tensorflow as tf

import plotly.express as px
import matplotlib.pyplot as plt


In [3]:
def generate_nn_coupling_tensor(D, L):
    """
    Vectorized nearest-neighbor coupling tensor for arbitrary D-dimensional lattice.
    Tensor shape: (L,)*D + (L,)*D
    J[i1,...,iD,j1,...,jD] = 1 if Manhattan distance = 1, else 0
    """
    N = L ** D

    # Generate all coordinates
    coords = np.array(np.meshgrid(*[np.arange(L)]*D, indexing='ij')).reshape(D, -1).T

    # Pairwise differences
    diff = coords[:, None, :] - coords[None, :, :]
    manhattan_dist = np.abs(diff).sum(axis=2)

    # Only nearest neighbors
    nn_mask = (manhattan_dist == 1)

    # Map to tensor
    tensor_shape = (L,)*D*2
    J_tensor = np.zeros(tensor_shape, dtype=int)

    for i in range(N):
        for j in range(N):
            if nn_mask[i,j]:
                J_tensor[tuple(coords[i]) + tuple(coords[j])] = 1

    return J_tensor

def generate_periodic_nn_coupling_tensor(D, L):
    """
    Vectorized nearest-neighbor coupling tensor with periodic boundaries.
    J[i1,...,iD,j1,...,jD] = 1 if periodic Manhattan distance = 1, else 0
    """
    # Generate all coordinates: shape (N, D)
    coords = np.array(np.meshgrid(*[np.arange(L)]*D, indexing='ij')).reshape(D, -1).T
    N = coords.shape[0]

    # Compute pairwise differences with broadcasting
    diff = np.abs(coords[:, None, :] - coords[None, :, :])
    
    # Apply periodic boundary
    diff = np.minimum(diff, L - diff)

    # Manhattan distance
    manhattan_dist = diff.sum(axis=2)

    # Nearest neighbors mask
    nn_mask = (manhattan_dist == 1)

    # Create empty tensor and set neighbors
    J_tensor = np.zeros((L,)*D*2, dtype=np.float32)
    
    # Get indices where nn_mask is True
    idx_i, idx_j = np.nonzero(nn_mask)
    
    # Set values in the tensor
    for i, j in zip(idx_i, idx_j):
        J_tensor[tuple(coords[i]) + tuple(coords[j])] = 1.0

    return J_tensor

import numpy as np

def generate_extended_coupling_tensor(D, L, Jnn, Jsn, Jdn):
    """
    Coupling tensor with periodic boundaries including:
    - Nearest neighbors (Manhattan distance = 1): ferromagnetic (+1)
    - Second neighbors (Manhattan distance = 2 along one axis only): antiferromagnetic (-1)
    - First diagonal neighbors (two coordinates differ by 1, others 0): antiferromagnetic (-1)
    """
    # Generate all coordinates: shape (N, D)
    coords = np.array(np.meshgrid(*[np.arange(L)]*D, indexing='ij')).reshape(D, -1).T
    N = coords.shape[0]

    # Pairwise differences with broadcasting
    diff = np.abs(coords[:, None, :] - coords[None, :, :])
    
    # Periodic boundary
    diff = np.minimum(diff, L - diff)

    # Manhattan distance
    manhattan_dist = diff.sum(axis=2)

    # ---- Masks ----
    nn_mask = (manhattan_dist == 1)   # first neighbors
    second_mask = (manhattan_dist == 2) & (diff.max(axis=2) == 2)   # second neighbors (2 in one axis)
    diag_mask = (manhattan_dist == 2) & (diff.max(axis=2) == 1)     # diagonal neighbors (two axes with 1 each)

    # Create tensor
    J_tensor = np.zeros((L,)*D*2, dtype=np.float32)

    # Fill ferromagnetic couplings (+1), 
    idx_i, idx_j = np.nonzero(nn_mask)
    for i, j in zip(idx_i, idx_j):
        J_tensor[tuple(coords[i]) + tuple(coords[j])] = Jnn

    # Fill antiferromagnetic couplings (-1) for second neighbors
    idx_i, idx_j = np.nonzero(second_mask)
    for i, j in zip(idx_i, idx_j):
        J_tensor[tuple(coords[i]) + tuple(coords[j])] = Jsn

    # Fill antiferromagnetic couplings (-1) for diagonal neighbors
    idx_i, idx_j = np.nonzero(diag_mask)
    for i, j in zip(idx_i, idx_j):
        J_tensor[tuple(coords[i]) + tuple(coords[j])] = Jdn

    return J_tensor


def run_temperature_sweep(D, L, betas, J=None, steps=None, accepted_steps=None, dynamic_theta = True, spherical = True, ising = False,theta_max=0.1):
    results = []

    for beta in betas:
        
        system = SpinSystem(D, L, J=J, keep_history=True, spherical=spherical, ising=ising)
        if dynamic_theta == True and ising == False:
            theta_max = theta_max/beta
            print(f"Sweeping with beta {beta} with max rotation of {theta_max} radian")
        else: 
            print(f"Sweeping with beta {beta}")
        system.metropolis_sweep(beta, steps=steps, accepted_steps=accepted_steps, theta_max=theta_max)

        df = pd.DataFrame(system.history.get()).T
        df["beta"] = beta
        results.append(df)

    return pd.concat(results, ignore_index=True)

In [8]:
D = 3
L = 100
N = L ** D


# J_nn = generate_nn_coupling_tensor(D, L)
J = generate_extended_coupling_tensor(D, L, 8, -1, -1) # (8, -1, -1):[5], ()

MemoryError: Unable to allocate 21.8 TiB for an array with shape (1000000, 1000000, 3) and data type int64

In [5]:
# betas = [0.1, 10, 100, 1000]
betas = [10, 1, 0.1]
df = run_temperature_sweep(D, L, betas, J = J, accepted_steps=1000, dynamic_theta=False, spherical=False, ising=True)

Sweeping with beta 10
Metropolis sweep: 1000/3152 accepted (31.73%)
Sweeping with beta 1
Metropolis sweep: 1000/2948 accepted (33.92%)
Sweeping with beta 0.1
Metropolis sweep: 1000/2358 accepted (42.41%)


In [7]:
df['abs_magnetization'] = df['magnetization'].abs()

df_accepted = df[df['accepted'] == True].copy()

df_accepted['step'] = df_accepted.groupby('beta').cumcount()

fig = px.line(
    df_accepted,
    x='step',
    y='abs_magnetization',
    color='beta',
    labels={
        'step': 'Step',
        'abs_magnetization': 'Magnetization ⟨s⟩',
        'beta': 'β'
    },
    title='Magnetization evolution at different temperatures'
)

fig.update_layout(
    template='plotly_white',
    legend_title='β',
    width=500,
    height=500
)

fig.show()